In [1]:
# импортирую необходимые библиотеки
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:/Users/Olga/Desktop/GIS/data.csv.gz', sep=';')

In [3]:
df.head()

,ID,ACCOUNTID,START_TIME,END_TIME,ZONENUMBER
0,374963901,6709301.0,1609448416165,1609449496165,c7ad
1,374964453,6170192.0,1609449343267,1609449463267,2745
2,374964422,6170567.0,1609449297411,1609449417411,8ee8
3,374964538,6709301.0,1609449459444,1609449579444,a54b
4,374964600,6170207.0,1609449549135,1609449669135,7468


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67237968 entries, 0 to 67237967
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   ID          int64  
 1   ACCOUNTID   float64
 2   START_TIME  int64  
 3   END_TIME    int64  
 4   ZONENUMBER  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 2.5+ GB


In [5]:
# поменяю формат у дат, чтобы их можно было использовать в расчетах
df['START_TIME']= pd.to_datetime(df['START_TIME'], origin='unix', unit='ms')
df['END_TIME']= pd.to_datetime(df['END_TIME'], origin='unix', unit='ms')
df.head()

,ID,ACCOUNTID,START_TIME,END_TIME,ZONENUMBER
0,374963901,6709301.0,2020-12-31 21:00:16.165,2020-12-31 21:18:16.165,c7ad
1,374964453,6170192.0,2020-12-31 21:15:43.267,2020-12-31 21:17:43.267,2745
2,374964422,6170567.0,2020-12-31 21:14:57.411,2020-12-31 21:16:57.411,8ee8
3,374964538,6709301.0,2020-12-31 21:17:39.444,2020-12-31 21:19:39.444,a54b
4,374964600,6170207.0,2020-12-31 21:19:09.135,2020-12-31 21:21:09.135,7468


In [6]:
# проверяю пропуски в данных
df.isnull().sum ()

ID                 0
ACCOUNTID     456842
START_TIME         0
END_TIME           0
ZONENUMBER         0
dtype: int64

In [7]:
# оставляю данные без пропусков, чтобы не исказить следующие расчеты
df= df[~df['ACCOUNTID'].isnull()]

In [8]:
df.isnull().sum ()

ID            0
ACCOUNTID     0
START_TIME    0
END_TIME      0
ZONENUMBER    0
dtype: int64

In [9]:
# оставляю данные за январь и февраль 2021 года
df = df[np.logical_and(df['START_TIME'] >= "2021-01-01", df['START_TIME'] <= "2021-02-28")]
df = df[np.logical_and(df['END_TIME'] >= "2021-01-01", df['END_TIME'] <= "2021-02-28")]

In [10]:
df.head()

,ID,ACCOUNTID,START_TIME,END_TIME,ZONENUMBER
541,374967021,6170251.0,2021-01-01 00:02:16.928,2021-01-01 00:04:16.928,bb8a
544,374967045,3122050.0,2021-01-01 00:04:30.425,2021-01-01 01:04:30.425,01c7
550,374967037,6709301.0,2021-01-01 00:03:54.054,2021-01-01 00:10:54.054,c4dc
554,374967064,6170245.0,2021-01-01 00:06:43.210,2021-01-01 01:06:43.210,480f
568,374967078,6202087.0,2021-01-01 00:08:26.179,2021-01-01 01:08:26.179,9788


In [11]:
# вычисляю длительность стоянки
df['duration'] = df['END_TIME'] - df['START_TIME']

In [12]:
df.head()

,ID,ACCOUNTID,START_TIME,END_TIME,ZONENUMBER,duration
541,374967021,6170251.0,2021-01-01 00:02:16.928,2021-01-01 00:04:16.928,bb8a,0 days 00:02:00
544,374967045,3122050.0,2021-01-01 00:04:30.425,2021-01-01 01:04:30.425,01c7,0 days 01:00:00
550,374967037,6709301.0,2021-01-01 00:03:54.054,2021-01-01 00:10:54.054,c4dc,0 days 00:07:00
554,374967064,6170245.0,2021-01-01 00:06:43.210,2021-01-01 01:06:43.210,480f,0 days 01:00:00
568,374967078,6202087.0,2021-01-01 00:08:26.179,2021-01-01 01:08:26.179,9788,0 days 01:00:00


In [13]:
# группирую данные по пользователю и зоне, суммирую длительность стоянки и вычисляю количество уникальных дней
df_1= df.groupby(['ACCOUNTID','ZONENUMBER']).agg({'duration': 'sum', 'START_TIME': 'nunique'})

In [14]:
df_1.head()

duration  START_TIME
ACCOUNTID ZONENUMBER                            
1.0       29b1       0 days 00:01:00           1
8.0       19b0       0 days 01:36:00           4
          fa4c       0 days 02:19:00           2
12.0      19b0       0 days 00:12:00           1
          4041       0 days 00:11:00           1

In [15]:
# нахожу для каждого пользователя любимую зону
favorite_zone = df_1[(df_1['duration'] > "05:00:00")]

In [16]:
favorite_zone.head()

duration  START_TIME
ACCOUNTID ZONENUMBER                            
65.0      ccac       0 days 06:56:00           8
221.0     8ee8       0 days 10:16:00           4
242.0     a444       0 days 08:00:00           3
          a54b       0 days 06:00:00           4
250.0     a77f       0 days 09:58:00          18

In [17]:
# вычисляю для пользователя - любимая зона регулярность
regular_stop = favorite_zone[favorite_zone['START_TIME'] > 15]
regular_stop.reset_index(inplace=True)

In [18]:
regular_stop.head()

,ACCOUNTID,ZONENUMBER,duration,START_TIME
0,250.0,a77f,0 days 09:58:00,18
1,333.0,df8e,1 days 05:49:00,19
2,354.0,df8e,0 days 10:50:00,16
3,412.0,19b0,0 days 10:11:00,25
4,564.0,fa4c,0 days 10:28:00,26


In [19]:
# оставляю данные за один день (15.01.2021) для ответа на 3 вопрос
df_one_day = df[(df['START_TIME'] >= '2021-01-15 00:00:00') & (df['START_TIME'] < '2021-01-16 00:00:00')]
df_one_day = df_one_day[(df_one_day['END_TIME'] >= '2021-01-15 00:00:00') & (df_one_day['END_TIME'] < '2021-01-16 00:00:00')]
df_one_day

,ID,ACCOUNTID,START_TIME,END_TIME,ZONENUMBER,duration
650219,376960881,7607815.0,2021-01-15 00:02:20.933,2021-01-15 00:32:20.933,19c2,0 days 00:30:00
650221,376960871,7918826.0,2021-01-15 00:01:17.065,2021-01-15 01:01:17.065,c7ad,0 days 01:00:00
650230,376960856,8081766.0,2021-01-15 00:00:15.896,2021-01-15 00:04:15.896,480f,0 days 00:04:00
650233,376960868,5739099.0,2021-01-15 00:01:07.830,2021-01-15 00:31:07.830,f65d,0 days 00:30:00
650236,376960867,5739099.0,2021-01-15 00:01:06.740,2021-01-15 00:05:06.740,71a6,0 days 00:04:00
...,...,...,...,...,...,...
1009660,377347764,1537526.0,2021-01-15 19:39:17.627,2021-01-15 20:00:17.627,e76a,0 days 00:21:00
1009661,377315880,1251736.0,2021-01-15 17:20:52.261,2021-01-15 20:30:52.261,19b0,0 days 03:10:00
1009662,377350241,1442685.0,2021-01-15 11:13:09.286,2021-01-15 11:58:09.286,431e,0 days 00:45:00
1009663,377350205,1442685.0,2021-01-15 19:19:34.535,2021-01-15 19:49:34.535,2634,0 days 00:30:00


In [20]:
# создаю столбец со часом
df_one_day['hour'] = df_one_day['END_TIME'].dt.hour
df_one_day.head()

,ID,ACCOUNTID,START_TIME,END_TIME,ZONENUMBER,duration,hour
650219,376960881,7607815.0,2021-01-15 00:02:20.933,2021-01-15 00:32:20.933,19c2,0 days 00:30:00,0
650221,376960871,7918826.0,2021-01-15 00:01:17.065,2021-01-15 01:01:17.065,c7ad,0 days 01:00:00,1
650230,376960856,8081766.0,2021-01-15 00:00:15.896,2021-01-15 00:04:15.896,480f,0 days 00:04:00,0
650233,376960868,5739099.0,2021-01-15 00:01:07.830,2021-01-15 00:31:07.830,f65d,0 days 00:30:00,0
650236,376960867,5739099.0,2021-01-15 00:01:06.740,2021-01-15 00:05:06.740,71a6,0 days 00:04:00,0


In [21]:
# вычисляю для каждого часа-зоны сумма всех времен
one_day_all = df_one_day.groupby(['hour', 'ZONENUMBER']).agg({'duration':'sum'})
one_day_all.reset_index(inplace=True)
one_day_all = one_day_all.rename(columns={'ZONENUMBER': 'zone_number'})
one_day_all

,hour,zone_number,duration
0,0,0432,0 days 00:30:00
1,0,077e,0 days 00:14:00
2,0,0a80,0 days 00:07:00
3,0,0bd2,0 days 00:38:00
4,0,1375,0 days 00:13:00
...,...,...,...
3238,23,f83b,0 days 01:00:00
3239,23,f847,0 days 00:56:00
3240,23,f9bf,0 days 10:53:00
3241,23,fa4c,0 days 16:54:00


In [22]:
# отфильтровываю из данных за один день регулярщиков
dff_reg = df_one_day[df_one_day['ACCOUNTID'].isin(regular_stop['ACCOUNTID'])].rename(columns={'ZONENUMBER': 'zone_number'})

In [23]:
dff_reg.head()

,ID,ACCOUNTID,START_TIME,END_TIME,zone_number,duration,hour
650219,376960881,7607815.0,2021-01-15 00:02:20.933,2021-01-15 00:32:20.933,19c2,0 days 00:30:00,0
650233,376960868,5739099.0,2021-01-15 00:01:07.830,2021-01-15 00:31:07.830,f65d,0 days 00:30:00,0
650236,376960867,5739099.0,2021-01-15 00:01:06.740,2021-01-15 00:05:06.740,71a6,0 days 00:04:00,0
650253,376960884,5739099.0,2021-01-15 00:03:03.681,2021-01-15 00:33:03.681,4d96,0 days 00:30:00,0
650267,376960942,5739099.0,2021-01-15 00:09:28.840,2021-01-15 00:39:28.840,e76a,0 days 00:30:00,0


In [24]:
# вычисляю для каждого часа-зоны сумма времени регулярщиков
dff_reg_group = dff_reg.groupby(['hour', 'zone_number']).agg({'duration':'sum'})
dff_reg_group.reset_index(inplace=True)
dff_reg_group = dff_reg_group.rename(columns={'duration': 'reg_time'})
dff_reg_group

,hour,zone_number,reg_time
0,0,0432,0 days 00:25:00
1,0,0a80,0 days 00:07:00
2,0,19c2,0 days 00:30:00
3,0,2634,0 days 00:54:00
4,0,2745,0 days 00:30:00
...,...,...,...
2557,23,f043,0 days 00:56:00
2558,23,f2ef,0 days 00:02:00
2559,23,f847,0 days 00:05:00
2560,23,fa4c,0 days 02:52:00


In [25]:
# соединяю таблицу всех и регурящиков в одну
finish = one_day_all.merge(dff_reg_group[['hour','zone_number','reg_time']], on=['hour','zone_number'])
finish

,hour,zone_number,duration,reg_time
0,0,0432,0 days 00:30:00,0 days 00:25:00
1,0,0a80,0 days 00:07:00,0 days 00:07:00
2,0,19c2,0 days 00:30:00,0 days 00:30:00
3,0,2634,0 days 01:10:00,0 days 00:54:00
4,0,2745,0 days 00:30:00,0 days 00:30:00
...,...,...,...,...
2557,23,f043,0 days 02:47:00,0 days 00:56:00
2558,23,f2ef,0 days 04:37:00,0 days 00:02:00
2559,23,f847,0 days 00:56:00,0 days 00:05:00
2560,23,fa4c,0 days 16:54:00,0 days 02:52:00


In [26]:
# вычисляю долю регулярщиков от всех
finish['ratio'] = round(finish['reg_time'] / finish['duration'] *100,2)

In [27]:
finish

,hour,zone_number,duration,reg_time,ratio
0,0,0432,0 days 00:30:00,0 days 00:25:00,83.33
1,0,0a80,0 days 00:07:00,0 days 00:07:00,100.00
2,0,19c2,0 days 00:30:00,0 days 00:30:00,100.00
3,0,2634,0 days 01:10:00,0 days 00:54:00,77.14
4,0,2745,0 days 00:30:00,0 days 00:30:00,100.00
...,...,...,...,...,...
2557,23,f043,0 days 02:47:00,0 days 00:56:00,33.53
2558,23,f2ef,0 days 04:37:00,0 days 00:02:00,0.72
2559,23,f847,0 days 00:56:00,0 days 00:05:00,8.93
2560,23,fa4c,0 days 16:54:00,0 days 02:52:00,16.96
